In [1]:
import cv2
import numpy as np
import requests
import imutils
from keras.models import load_model

In [2]:
model = load_model('./new_model77.h5')

In [3]:
gestures = {
    1:'A',
    2:'B',
    3:'C',
    4:'D',
    5:'E',
    6:'F',
    7:'G',
    8:'H',
    9:'I',
    10:'K',
    11:'L',
    12:'M',
    13:'N',
    14:'O',
    15:'P',
    16:'Q',
    17:'R',
    18:'S',
    19:'T',
    20:'U',
    21:'V',
    22:'W',
    23:'X',
    24:'Y',
}


In [4]:
def predict(gesture):
    img = cv2.resize(gesture, (50,50))
    img = img.reshape(1,50,50,1)
    img = img/255.0
    prd = model.predict(img)
    index = prd.argmax()
    return gestures[index]

In [5]:
def ip_cam(address):
    # Replace the below URL with your own url of ip cam. Make sure to add "/shot.jpg" at last.
   # url = "http://10.7.34.197:8080/shot.jpg"

    # While loop to continuously fetching data from the Url of mobile camera
    while True:

        img_resp = requests.get(address)

        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        img = cv2.imdecode(img_arr, 1)
        img = cv2.flip(img, -1)

        return img

In [6]:
#vc = cv2.VideoCapture(0)
#rval, frame = vc.read()
 # Replace the below URL with your own. Make sure to add "/shot.jpg" at last.
url = "http://10.51.171.240:8080/shot.jpg"
frame = ip_cam(url)
old_text = ''
pred_text = ''
sign_pic = cv2.imread("pic_sign.png")
pic= cv2.resize(sign_pic,(300,300))
count_frames = 0
total_str = ''
flag = False



while True:
    
    if frame is not None: 
        
        
        frame = cv2.resize( frame, (500,400) )
        
        
        cv2.rectangle(frame, (300,300), (100,100), (0,255,0), 2)
        
        crop_img = frame[100:300, 100:300]
        grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        
        thresh = cv2.threshold(grey,210,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
      
        
        blackboard = np.zeros(frame.shape, dtype=np.uint8)
        cv2.putText(blackboard, "Predicted Text--> " + pred_text, (20, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 255, 0))


        if count_frames > 20 and pred_text != "":
            total_str += pred_text
            count_frames = 0
            
        if flag == True:
            old_text = pred_text
            pred_text = predict(thresh)
        
            if old_text == pred_text:
                count_frames += 1
            else:
                count_frames = 0
            cv2.putText(blackboard, total_str, (20,130), cv2.FONT_ITALIC, 1.2, (255, 25, 127))
        res = np.hstack((frame, blackboard))
        
        cv2.imshow("Image", res)
        cv2.imshow("Hand", thresh)
        cv2.imshow("Sign Demo", pic)
        
    frame = ip_cam(url)
    keypress = cv2.waitKey(1)
    if keypress == ord('c'):
        flag = True
    if keypress == ord('q'):
        break


cv2.destroyAllWindows()
cv2.waitKey(1)



-1